# Data Preprocessing

In [11]:
# Imports
import pandas as pd
import csv
import bs4 as BeautifulSoup
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import sklearn
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import nba_api.stats.endpoints
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nba_api.stats.endpoints import playergamelog
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'selenium'

In [15]:
# Karl-Anthony Towns ID
KAT_ID = 1626157

# Example: fetch 2022-23 regular season data
kat_log = playergamelog.PlayerGameLog(player_id=KAT_ID, season='2023-24', season_type_all_star='Regular Season')
kat_df = kat_log.get_data_frames()[0]

print(kat_df.columns)
kat_df.head()

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1626157,0022301194,"APR 14, 2024",MIN vs. PHX,L,29,3,8,0.375,...,4,6,2,0,0,5,4,10,-18,1
1,22023,1626157,0022301178,"APR 12, 2024",MIN vs. ATL,W,28,4,11,0.364,...,4,5,8,0,1,3,2,11,-4,1
2,22023,1626157,0022300881,"MAR 04, 2024",MIN vs. POR,W,21,5,12,0.417,...,2,4,1,0,2,1,4,14,1,1
3,22023,1626157,0022300873,"MAR 03, 2024",MIN vs. LAC,L,34,7,18,0.389,...,5,9,8,0,0,3,5,18,14,1
4,22023,1626157,0022300862,"MAR 01, 2024",MIN vs. SAC,L,37,7,17,0.412,...,4,6,5,1,0,3,6,19,1,1


Creating the list of dates that KAT played from the NBA_API

In [16]:
# Verifying that KAT played 62 games
kat_df['GAME_DATE']

0     APR 14, 2024
1     APR 12, 2024
2     MAR 04, 2024
3     MAR 03, 2024
4     MAR 01, 2024
          ...     
57    NOV 04, 2023
58    NOV 01, 2023
59    OCT 30, 2023
60    OCT 28, 2023
61    OCT 25, 2023
Name: GAME_DATE, Length: 62, dtype: object

In [17]:
date_series = kat_df['GAME_DATE']

# Convert the series of date strings to datetime
kat_game_date = pd.to_datetime(date_series, format='%b %d, %Y')
kat_game_date = kat_game_date.sort_values().reset_index(drop=True)
print(kat_game_date)

0    2023-10-25
1    2023-10-28
2    2023-10-30
3    2023-11-01
4    2023-11-04
        ...    
57   2024-03-01
58   2024-03-03
59   2024-03-04
60   2024-04-12
61   2024-04-14
Name: GAME_DATE, Length: 62, dtype: datetime64[ns]


In [12]:
# Set up Selenium WebDriver
driver = webdriver.Chrome()


url = "https://www.bettingpros.com/nba/odds/player-props/karl-anthony-towns/?date=2024-04-14"
driver.get(url)


xpath = "//*[@id='mainContent']/div/section/div/section/div[12]/div[1]/div[7]/button[1]/div/div/div/div/span[1]"

try:
    # Wait for the element to load
    element = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )

    
    inner_text = driver.execute_script("return arguments[0].innerText;", element)
    print("Extracted Inner Text:", inner_text)

except Exception as e:
    print(f"Error: {e}")

finally:
    
    driver.quit()


NameError: name 'webdriver' is not defined

In [18]:
# Getting the column for the player defense vs position
position_defense_url = "https://www.fantasypros.com/nba/defense-vs-position.php?year=2023"
position_headers = {"User-Agent": "Mozilla/5.0"}
position_response = requests.get(url, headers=position_headers)
position_soup = BeautifulSoup(position_response.text, "html.parser")
# Finds the first table with data and extracts the rows
table = position_soup.find("table", {"class": "data"})
rows = table.find_all("tr")
# Extract the headers of the columns
headers = [th.text.strip() for th in rows[0].find_all("th")]
data = []
for row in rows[1:]:
    cols = row.find_all("td")
    if len(cols) > 0:
        data.append([col.text.strip() for col in cols])
#Create a dataframe with the data
position_df = pd.DataFrame(data, columns=headers)
for col in position_df.columns[1:]:
    position_df[col] = pd.to_numeric(position_df[col], errors='coerce')
#Positions
positions = ["PG", "SG", "SF", "PF", "C"]
ranked_data = {}
for pos in positions:
    if f"{pos} REB" in position_df.columns:
        ranked_data[pos] = position_df[["Team", f"{pos} REB"]].dropna().sort_values(by=f"{pos} REB", ascending=False)
        ranked_data[pos].reset_index(drop=True, inplace = True)
        ranked_data[pos]["Rank"] = ranked_data[pos].index + 1 #Put a ranking column to order teams
#Display rankings
for pos in positions:
    print(f"\n--- {pos} Rebounds Rankings ---")
    print(ranked_data[pos])

NameError: name 'requests' is not defined

In [21]:
# Set up Selenium WebDriver
driver = webdriver.Chrome()

# Base URL
base_url = "https://www.bettingpros.com/nba/odds/player-props/karl-anthony-towns/?date="

# XPath for the element
xpath = "//*[@id='mainContent']/div/section/div/section/div[12]/div[1]/div[7]/button[1]/div/div/div/div/span[1]"

# Dates KAT played (replace with your `kat_game_date` DataFrame/Series)
kat_game_date  # Example dates

# Store results
results = []

try:
    for date in kat_game_date:
        # Construct the URL for the specific date
        url = f"{base_url}{date}"
        driver.get(url)

        try:
            # Wait for the element to load
            element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )

            # Extract the innerText
            inner_text = driver.execute_script("return arguments[0].innerText;", element)
            print(f"Date: {date}, Extracted Inner Text: {inner_text}")

            # Append the result to the list
            results.append({"date": date, "line": inner_text})

        except Exception as e:
            print(f"Error on date {date}: {e}")
            # Append None if there's an error
            results.append({"date": date, "line": None})

        # Politely wait to avoid overloading the server
        time.sleep(5)  # Adjust as needed (5 seconds is reasonable)

finally:
    # Close the browser
    driver.quit()

# Convert results to a DataFrame for further analysis or saving
results_df = pd.DataFrame(results)
print(results_df)

# Save results to a CSV file
results_df.to_csv("KAT_rebounding_lines.csv", index=False)


Error on date 2023-10-25 00:00:00: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF71AF902F5+28725]
	(No symbol) [0x00007FF71AEF2AE0]
	(No symbol) [0x00007FF71AD8510A]
	(No symbol) [0x00007FF71ADD93D2]
	(No symbol) [0x00007FF71ADD95FC]
	(No symbol) [0x00007FF71AE23407]
	(No symbol) [0x00007FF71ADFFFEF]
	(No symbol) [0x00007FF71AE20181]
	(No symbol) [0x00007FF71ADFFD53]
	(No symbol) [0x00007FF71ADCA0E3]
	(No symbol) [0x00007FF71ADCB471]
	GetHandleVerifier [0x00007FF71B2BF30D+3366989]
	GetHandleVerifier [0x00007FF71B2D12F0+3440688]
	GetHandleVerifier [0x00007FF71B2C78FD+3401277]
	GetHandleVerifier [0x00007FF71B05AAAB+858091]
	(No symbol) [0x00007FF71AEFE74F]
	(No symbol) [0x00007FF71AEFA304]
	(No symbol) [0x00007FF71AEFA49D]
	(No symbol) [0x00007FF71AEE8B69]
	BaseThreadInitThunk [0x00007FFD1DEA259D+29]
	RtlUserThreadStart [0x00007FFD1E98AF38+40]

Error on date 2023-10-28 00:00:00: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF71AF902F5+28725]
	(No symbol) [0x00007FF71AEF2AE0]
	(N

KeyboardInterrupt: 